In [1]:
import sys
!{sys.executable} -m pip install opencv-python IPython ffmpeg tqdm google-generativeai google cv2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 1.6 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: /Users/shy/Projects/clippy/env/bin/python -m pip install --upgrade pip


In [2]:
from pathlib import Path
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown
import cv2
import shutil
import ffmpeg
import subprocess
import os
from tqdm import tqdm

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

/Users/shy/Projects/clippy/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os

genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

In [4]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)


models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [5]:
model = genai.GenerativeModel('gemini-pro')

In [6]:
response = model.generate_content("What is the meaning of life?")

In [7]:
print(response)
to_markdown(response.text)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'finish_reason': 4, 'index': 0, 'safety_ratings': [], 'token_count': 0, 'grounding_attributions': []}]}),
)


ValueError: The `response.text` quick accessor only works when the response contains a valid `Part`, but none was returned. Check the `candidate.safety_ratings` to see if the response was blocked.

In [ ]:
response.prompt_feedback

In [ ]:
response.candidates


In [ ]:
response = model.generate_content("hi man how are you", stream=True)

## Pro-Vision Video!

In [8]:
model = genai.GenerativeModel('gemini-pro-vision')


In [9]:
!ls /Users/shy/Desktop/4-19-Sasha-blocked.MP4   

/Users/shy/Desktop/4-19-Sasha-blocked.MP4


In [10]:
FRAME_EXTRACTION_DIRECTORY = "frames"
FRAME_PREFIX = "_frame"

video_file_name = Path("/Users/shy/Desktop/4-19-Sasha-blocked.MP4")
video_file_name = Path("/Users/shy/Desktop/4-19-Sasha-dinner.MP4")
video_file_name = Path("/Users/shy/Desktop/test_clip.mp4")
video_file_name = Path("/Users/shy/Desktop/intros.mp4")

audio_file_name = Path(f"{FRAME_EXTRACTION_DIRECTORY}/{video_file_name.stem}.mp3")

def call_ffmpeg(arg_string):
    subprocess.call(['ffmpeg', "-hide_banner", "-loglevel", "error", "-y"] + arg_string.split())
    
def extract_mp3_from_mp4(video_file_name, audio_file_name):
    call_ffmpeg(f"-i {video_file_name} {audio_file_name}")

def convert_to_mp4(video_file_name):
    mkv_name = video_file_name.with_suffix(".mkv")
    call_ffmpeg(f"-i {video_file_name} -c copy {mkv_name}")


extract_mp3_from_mp4(video_file_name, audio_file_name)
convert_to_mp4(video_file_name)

audio_file = genai.upload_file(path=audio_file_name)

In [11]:
class File:
  def __init__(self, file_path: str, display_name: str = None):
    self.file_path = file_path
    if display_name:
      self.display_name = display_name
    self.timestamp = get_timestamp(file_path)

  def set_file_response(self, response):
    self.response = response

def get_timestamp(filename):
  """Extracts the frame count (as an integer) from a filename with the format
     'output_file_prefix_frame00:00.jpg'.
  """
  parts = filename.split(FRAME_PREFIX)
  if len(parts) != 2:
      return None  # Indicates the filename might be incorrectly formatted
  return parts[1].split('.')[0]

# Process each frame in the output directory
files = os.listdir(FRAME_EXTRACTION_DIRECTORY)
files = sorted(files)
files_to_upload = []
for file in files:
  if not file.endswith('.jpg'):
    continue
  files_to_upload.append(
      File(file_path=os.path.join(FRAME_EXTRACTION_DIRECTORY, file)))

# Upload the files to the API
uploaded_files = []
print(f'Uploading {len(files_to_upload)} files. This might take a bit...')

for file in tqdm(files_to_upload):
  print(f'Uploading: {file.file_path}...')
  response = genai.upload_file(path=file.file_path)
  file.set_file_response(response)
  uploaded_files.append(file)

print(f"Completed file uploads!\n\nUploaded: {len(uploaded_files)} files")
     

Uploading 285 files. This might take a bit...


  0%|          | 0/285 [00:00<?, ?it/s]

Uploading: frames/intros_mp4_frame00:00.jpg...


  0%|          | 1/285 [00:01<05:01,  1.06s/it]

Uploading: frames/intros_mp4_frame00:01.jpg...


...

  5%|▍         | 13/285 [00:14<04:59,  1.10s/it]

Uploading: frames/intros_mp4_frame00:13.jpg...


  5%|▍         | 14/285 [00:15<04:55,  1.09s/it]

Uploading: frames/intros_mp4_frame00:14.jpg...


  5%|▌         | 15/285 [00:16<04:55,  1.10s/it]

In [ ]:
# List files uploaded in the API
for n, f in zip(range(len(uploaded_files)), genai.list_files()):
  print(f.uri)

https://generativelanguage.googleapis.com/v1beta/files/azdrk4nqp6h4
https://generativelanguage.googleapis.com/v1beta/files/441sdko7004y
https://generativelanguage.googleapis.com/v1beta/files/ww6t49xai1aq
https://generativelanguage.googleapis.com/v1beta/files/kex2mt6fj95q
https://generativelanguage.googleapis.com/v1beta/files/y6dqer0o1iu
https://generativelanguage.googleapis.com/v1beta/files/nbl1mnisnh3x
https://generativelanguage.googleapis.com/v1beta/files/ipks3dpdtbe0
https://generativelanguage.googleapis.com/v1beta/files/yxbcndrk75gw
https://generativelanguage.googleapis.com/v1beta/files/7lm7qjj8wuz7
https://generativelanguage.googleapis.com/v1beta/files/l7b0ucuerzo6
https://generativelanguage.googleapis.com/v1beta/files/26mv984nqckj
https://generativelanguage.googleapis.com/v1beta/files/k9ja26v70gdh
https://generativelanguage.googleapis.com/v1beta/files/9fj87gr7gepx
https://generativelanguage.googleapis.com/v1beta/files/3hgvph1fzmdj
https://generativelanguage.googleapis.com/v1beta/

In [ ]:

#Choose only clips that are entertaining or make for good b-roll.
prompt = """
You are an expert video editor, helping us select clips for a video.
Select only clips where someone tells us their name.
Double check your work to ensure that the timestamps are correct.

For each clip, provide a "start_timestamp", "end_timestamp", "location" (point of interest in the frame, either left, middle-left, middle, middle-right, or right), and "description" (a brief explanation of why you selected the clip).
"""

# Set the model to Gemini 1.5 Pro.
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest", generation_config={"response_mime_type": "application/json"})

# Make GenerateContent request with the structure described above.
def make_request(prompt, files):
  request = [prompt]
  for file in files:
    request.append(file.timestamp)
    print(file.timestamp)
    request.append(file.response)
  return request

# Make the LLM request.
request = make_request(prompt, uploaded_files) + [audio_file]
response = model.generate_content(request,
                                  request_options={"timeout": 600})
print(response.text)

[
  {
    "start_timestamp": "00:08",
    "end_timestamp": "00:09",
    "location": "middle",
    "description": "The interviewee identifies as a UCI student."
  },
  {
    "start_timestamp": "01:06",
    "end_timestamp": "01:07",
    "location": "middle-left",
    "description": "The interviewee states his name is Sasha."
  },
  {
    "start_timestamp": "01:07",
    "end_timestamp": "01:08",
    "location": "middle-right",
    "description": "The interviewee states his name is Michael."
  },
  {
    "start_timestamp": "02:12",
    "end_timestamp": "02:13",
    "location": "middle-left",
    "description": "The interviewee states her name is Katelyn."
  },
  {
    "start_timestamp": "02:15",
    "end_timestamp": "02:16",
    "location": "middle-left",
    "description": "The interviewer states his name is Sasha."
  },
  {
    "start_timestamp": "02:18",
    "end_timestamp": "02:21",
    "location": "middle",
    "description": "The interviewee states his name is Sam, clarifying pronunc

In [71]:
from json import loads

clip_details = loads(response.text)

def get_seconds(time_string):
    denoms = time_string.split(":")
    secs = 0
    for i, val in enumerate(denoms[::-1]):
        secs += float(val) * 60 ** i 
    return secs

def clip(start, end, mkv_filename, filename):
    call_ffmpeg(f"-ss {start} -i {mkv_filename} -c copy -t {end - start} {filename}")

for i, details in enumerate(clip_details, start=1):
    clip_name = f"frames/clip_{i}.mkv"
    clip(get_seconds(details['start_timestamp']), get_seconds(details['end_timestamp']), video_file_name, clip_name)

In [ ]:
#print(f'Deleting {len(uploaded_files)} images. This might take a bit...')
#for file in uploaded_files:
#  genai.delete_file(file.response.name)
#  print(f'Deleted {file.file_path} at URI {file.response.uri}')
#print(f"Completed deleting files!\n\nDeleted: {len(uploaded_files)} files")

In [37]:
import os

frames_directory = 'frames'
total_size = 0

for root, dirs, files in os.walk(frames_directory):
    for file in files:
        file_path = os.path.join(root, file)
        total_size += os.path.getsize(file_path)

print(f"The total size of all files in the frames directory is: {total_size} bytes")


The total size of all files in the frames directory is: 139088916 bytes
